In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict

/nfs/homedirs/fuchsgru/miniconda3/lib/python3.9/site-packages/seml/database.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
collection_name = 'week6_cora_full_ai_vs_os'
collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    with open(ex['result']) as f:
        ex['result'] = json.load(f)

print(f'Number of finished experiments : {len(experiments)}')

Number of finished experiments : 8


In [3]:
def get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True):
    exs = [
        r for r in experiments if r['config']['model']['residual'] == residual and r['config']['model']['use_spectral_norm'] == spectral_norm and r['config']['data']['train_labels_remove_other'] == train_labels_remove_other
    ]
    return exs[0]

In [4]:
experiment = get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True)

In [94]:

d = {}
for experiment in experiments:
    residual = experiment['config']['model']['residual']
    spectral_norm = experiment['config']['model']['use_spectral_norm']
    remove_other = experiment['config']['data']['train_labels_remove_other']
    for k, v in experiment['result'].items():
        mean, std = np.array(v).mean(), np.array(v).std()
        if 'auroc' in k:
            metric, subset = k.split('_subset-')
            if 'softmax_entropy' in metric:
                d[(remove_other, residual, spectral_norm, subset, 'softmax entropy', '', 'mean')] =  [mean]
                d[(remove_other, residual, spectral_norm, subset, 'softmax entropy', '', 'std')] =  [std]
                
            else:
                d[(remove_other, residual, spectral_norm, subset, 'feature density', metric.replace('auroc:', ''), 'mean')] = [mean]
                d[(remove_other, residual, spectral_norm, subset, 'feature density', metric.replace('auroc:', ''), 'std')] = [std]
        elif 'val_accuracy' in k:
            k = k.replace('val_accuracy', 'acc')
            d[(remove_other, residual, spectral_norm, '', k, '', 'mean')] = [mean]
            d[(remove_other, residual, spectral_norm, '', k, '', 'std')] = [std]
                
df = pd.DataFrame(d).T.sort_index()
df = df.reset_index((0, 1, 2, -1))

In [95]:
subdfs, names = [], []
for n, g in df.groupby(['level_0', 'level_1', 'level_2', 'level_6']):
    g = pd.DataFrame(g[0])
    g.columns = [n]
    subdfs.append(g)

In [100]:
df_cat = pd.concat(subdfs, axis=1)
idx = pd.MultiIndex.from_tuples(df_cat.columns, names=('Remove OOD', 'Residual', 'Spectral Norm', 'Stat'))
df_cat.columns = idx
df_cat.index.names = ('OOD', '', '')
pd.set_option("display.precision", 2)
df_cat = df_cat.T.sort_index().T
df_cat

Remove OOD                                  False                          \
Residual                                    False                   True    
Spectral Norm                               False       True        False   
Stat                                         mean   std  mean   std  mean   
OOD                                                                         
    acc-val-reduced                          0.79  0.03  0.79  0.03  0.79   
    acc-val-train-labels                     0.79  0.03  0.79  0.03  0.79   
ai  feature density      2-mog:16-isomap     0.54  0.14  0.53  0.06  0.50   
                         2-mog:24-isomap     0.64  0.09  0.59  0.05  0.56   
                         2-mog:no            0.65  0.05  0.72  0.04  0.61   
                         3-mog:16-isomap     0.60  0.14  0.57  0.06  0.59   
                         3-mog:24-isomap     0.68  0.06  0.61  0.04  0.62   
                         3-mog:no            0.67  0.07  0.71  0.03  0.57   
                         7-mog:16-isomap     0.70  0.04  0.61  0.05  0.64   
                         7-mog:24-isomap     0.71  0.04  0.60  0.06  0.67   
                         7-mog:no            0.68  0.10  0.69  0.03  0.61   
                         gpc-full:16-isomap  0.72  0.03  0.68  0.04  0.74   
                         gpc-full:24-isomap  0.72  0.03  0.72  0.03  0.73   
                         gpc-full:no         0.77  0.03  0.81  0.03  0.67   
    softmax entropy                          0.81  0.03  0.57  0.03  0.83   
os  feature density      2-mog:16-isomap     0.55  0.23  0.50  0.08  0.44   
                         2-mog:24-isomap     0.67  0.17  0.55  0.07  0.53   
                         2-mog:no            0.59  0.09  0.65  0.07  0.59   
                         3-mog:16-isomap     0.64  0.23  0.53  0.07  0.59   
                         3-mog:24-isomap     0.75  0.13  0.57  0.07  0.61   
                         3-mog:no            0.65  0.11  0.63  0.05  0.56   
                         7-mog:16-isomap     0.77  0.11  0.56  0.07  0.69   
                         7-mog:24-isomap     0.78  0.10  0.56  0.06  0.70   
                         7-mog:no            0.70  0.12  0.64  0.05  0.65   
                         gpc-full:16-isomap  0.80  0.08  0.65  0.03  0.77   
                         gpc-full:24-isomap  0.79  0.10  0.70  0.03  0.77   
                         gpc-full:no         0.81  0.09  0.81  0.07  0.73   
    softmax entropy                          0.93  0.04  0.67  0.03  0.94   

Remove OOD                                                    True         \
Residual                                                      False         
Spectral Norm                                     True        False         
Stat                                          std  mean   std  mean   std   
OOD                                                                         
    acc-val-reduced                          0.03  0.74  0.02  0.82  0.02   
    acc-val-train-labels                     0.03  0.74  0.02  0.83  0.02   
ai  feature density      2-mog:16-isomap     0.05  0.66  0.08  0.55  0.16   
                         2-mog:24-isomap     0.08  0.70  0.06  0.60  0.16   
                         2-mog:no            0.11  0.33  0.05  0.66  0.06   
                         3-mog:16-isomap     0.08  0.70  0.06  0.62  0.13   
                         3-mog:24-isomap     0.09  0.71  0.05  0.67  0.11   
                         3-mog:no            0.11  0.37  0.04  0.64  0.06   
                         7-mog:16-isomap     0.07  0.69  0.06  0.72  0.06   
                         7-mog:24-isomap     0.07  0.68  0.06  0.73  0.05   
                         7-mog:no            0.11  0.40  0.07  0.78  0.08   
                         gpc-full:16-isomap  0.03  0.72  0.05  0.75  0.05   
                         gpc-full:24-isomap  0.03  0.75  0.04  0.75  0.05   
                         gpc-full:no         0.07  0.62  0.06  0.80  0.0